# Multi-Account EC2 Cost Optimization Report

**Comprehensive analysis of EC2 spending across multiple AWS accounts**

This notebook provides actionable insights for Engineering and DevOps teams to optimize AWS EC2 costs across your organization.

---

## Configuration

In [1]:
# === CONFIGURATION ===
import glob
import os

# Auto-detect latest multi-account cost report
report_files = glob.glob("multi_account_cost_report_*.json") + glob.glob("data/multi_account_cost_report_*.json")
if report_files:
    REPORT_JSON_PATH = max(report_files, key=os.path.getmtime)
else:
    raise FileNotFoundError("No multi-account cost report found. Run: uv run python multi_account_analyzer.py")

DEFAULT_REGION = "eu-west-2"  # Default AWS region for commands
# === END CONFIGURATION ===

print(f"Using report: {REPORT_JSON_PATH}")

Using report: multi_account_cost_report_20251223_155151.json


## Setup & Imports

In [2]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import warnings
from datetime import datetime

# Import utility functions
from utils.report_utils import (
    # Multi-account functions
    load_multi_account_report, get_account_list, get_account_data, get_aggregated_data,
    parse_account_costs,
    create_account_cost_bar_chart, create_account_cost_pie_chart,
    create_multi_account_monthly_trend, create_multi_account_kpi_cards,
    create_multi_account_savings_waterfall,
    create_instance_type_stacked_by_component,
    # Single-account functions (for drill-down)
    parse_monthly_costs, parse_instance_type_costs, parse_region_costs, parse_tag_costs,
    parse_cloudwatch_instances, parse_ebs_analysis, parse_spot_analysis, parse_ri_sp_analysis,
    parse_fiscal_year_forecast, parse_service_costs,
    create_monthly_trend_chart, create_horizontal_bar_chart, create_pie_chart,
    create_gauge_chart, create_waterfall_chart, create_fiscal_year_forecast_chart,
    create_fiscal_year_summary_kpis,
    generate_stop_instance_command, generate_batch_stop_script,
    format_currency, format_percentage
)

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 50)

print("Libraries imported successfully")

Libraries imported successfully


## Load Data

In [3]:
# Load multi-account report
try:
    data = load_multi_account_report(REPORT_JSON_PATH)
    print(f"Loaded multi-account cost report: {REPORT_JSON_PATH}")
    print(f"Generated: {data['generated_at']}")
    print(f"Accounts analyzed: {data['accounts_analyzed']}")
    if data.get('accounts_failed', 0) > 0:
        print(f"Accounts failed: {data['accounts_failed']}")
    print("\nAccounts included:")
    for acc in get_account_list(data):
        print(f"  - {acc['account_name']}: ${acc['total_cost']:,.2f}")
except Exception as e:
    print(f"Error loading report: {e}")
    raise

Loaded multi-account cost report: multi_account_cost_report_20251223_155151.json
Generated: 2025-12-23T15:51:51.468985
Accounts analyzed: 7

Accounts included:
  - AISI-Research-Platform-prod: $624,671.91
  - AISI-Self-Hosting-prod: $56,652.02
  - AISI-Research-Platform-preprod: $13,930.89
  - AISI-Research-Platform-dev: $5,973.28
  - AISIWAProduction: $231.46
  - AISIWADevelopment: $25.48
  - AISIWAStaging: $0.01


---
# Section 1: Multi-Account Executive Summary

High-level overview of costs across all AWS accounts.

In [4]:
# Get aggregated data
aggregated = get_aggregated_data(data)

total_ec2_cost = aggregated.get('total_ec2_cost', 0)
total_aws_cost = aggregated.get('total_aws_cost', total_ec2_cost)
total_optimization = aggregated.get('total_optimization_potential', 0)
accounts_count = data.get('accounts_analyzed', 0)

print("\n" + "="*80)
print("MULTI-ACCOUNT EXECUTIVE SUMMARY")
print("="*80)
print(f"\nAccounts Analyzed: {accounts_count}")
print(f"Total EC2 Cost (3-month): ${total_ec2_cost:,.2f}")
print(f"Total AWS Cost (3-month): ${total_aws_cost:,.2f}")
print(f"Total Optimization Potential: ${total_optimization:,.2f}/month (${total_optimization*12:,.2f}/year)")
print("="*80)


MULTI-ACCOUNT EXECUTIVE SUMMARY

Accounts Analyzed: 7
Total EC2 Cost (3-month): $701,485.05
Total AWS Cost (3-month): $1,145,329.38
Total Optimization Potential: $115,209.48/month ($1,382,513.78/year)


In [5]:
# Multi-Account KPI Cards
fig = create_multi_account_kpi_cards(data)
fig.show()

## Cost Distribution by Account

In [6]:
# Cost by Account - Pie Chart
fig = create_account_cost_pie_chart(data, "Cost Distribution by Account")
fig.show()

## Fiscal Year Forecast

In [7]:
# Aggregated Fiscal Year Forecast
fy_data = aggregated.get('fiscal_year_forecast', {})

if fy_data:
    total_actual = fy_data.get('total_actual', 0)
    total_forecast = fy_data.get('total_forecast', 0)
    total_projected = fy_data.get('total_projected', 0)
    
    print("\n" + "="*80)
    print("AGGREGATED FISCAL YEAR FORECAST")
    print("="*80)
    print(f"\nActual Spend (YTD):      ${total_actual:>15,.2f}")
    print(f"Forecasted Remaining:    ${total_forecast:>15,.2f}")
    print(f"{'-'*50}")
    print(f"Projected FY Total:      ${total_projected:>15,.2f}")
    print("="*80)
else:
    print("No aggregated fiscal year forecast data available.")


AGGREGATED FISCAL YEAR FORECAST

Actual Spend (YTD):      $   3,987,785.06
Forecasted Remaining:    $   1,373,415.72
--------------------------------------------------
Projected FY Total:      $   5,361,200.78


---
# Section 2: Aggregated Cost Breakdown

Cost breakdown across all accounts.

## 2.1 Instance Types by Cost & Component

In [8]:
# Instance type costs stacked by Component tag
fig = create_instance_type_stacked_by_component(
    data, 
    top_n=15, 
    title='Top 15 Instance Types by Cost - Stacked by Component Tag'
)
fig.show()

---
# Section 2.5: Deep dive - Research Platform Prod

Detailed analysis of the highest-spending account (89% of total EC2 spend).

In [9]:
# Deep Dive: Research Platform Prod Account
# Account ID: 302263072849 (hardcoded as it's the highest spender - 89% of total EC2 cost)

TOP_ACCOUNT_ID = "302263072849"
top_account = get_account_data(data, TOP_ACCOUNT_ID)
top_account_name = top_account.get('account_name', TOP_ACCOUNT_ID)
top_account_cost = top_account.get('total_cost', 0)
top_account_action_plan = top_account.get('action_plan', {})
top_account_savings = top_account_action_plan.get('total_savings', 0)

# Calculate percentage of total
total_ec2 = aggregated.get('total_ec2_cost', 1)
cost_percentage = (top_account_cost / total_ec2 * 100)

print("\n" + "="*80)
print(f"DEEP DIVE: {top_account_name}")
print("="*80)
print(f"\nAccount ID: {TOP_ACCOUNT_ID}")
print(f"EC2 Cost (3-month): ${top_account_cost:>15,.2f}  ({cost_percentage:.1f}% of total)")
print(f"Optimization Potential: ${top_account_savings:>11,.2f}/month (${top_account_savings*12:,.2f}/year)")
print("="*80)


DEEP DIVE: AISI-Research-Platform-prod

Account ID: 302263072849
EC2 Cost (3-month): $     624,671.91  (89.0% of total)
Optimization Potential: $ 106,315.43/month ($1,275,785.21/year)


### 2.5.1 Instance Type Analysis

Detailed breakdown of compute instance types driving costs in Research Platform Prod.

In [10]:
# Instance type breakdown for Research Platform Prod
top_instance_df = parse_instance_type_costs(top_account, top_n=15)

if not top_instance_df.empty:
    # Create bar chart
    fig = create_horizontal_bar_chart(
        top_instance_df, 'cost', 'instance_type',
        f'Top 15 Instance Types - {top_account_name}', 'Cost ($)'
    )
    fig.update_layout(height=500)
    fig.show()
    
    # Get all instance types (not just top 15) for category analysis
    all_instance_df = parse_instance_type_costs(top_account, top_n=100)
    
    # Categorize instance types - separate P (accelerated compute) and G (graphics/inference)
    p_instances = all_instance_df[all_instance_df['instance_type'].str.match(r'^p\d', na=False)]
    g_instances = all_instance_df[all_instance_df['instance_type'].str.match(r'^g\d', na=False)]
    storage_instances = all_instance_df[all_instance_df['instance_type'].str.match(r'^(i\d|d\d)', na=False)]
    compute_instances = all_instance_df[all_instance_df['instance_type'].str.match(r'^(c\d|z\d)', na=False)]
    general_instances = all_instance_df[all_instance_df['instance_type'].str.match(r'^(m\d|t\d|a\d|r\d)', na=False)]
    
    print("\nCost by Instance Category:")
    print("-" * 60)
    if not p_instances.empty:
        p_total = p_instances['cost'].sum()
        print(f"  P-series (ML Training/HPC):    ${p_total:>12,.2f} ({p_total/top_account_cost*100:.1f}%)")
    if not g_instances.empty:
        g_total = g_instances['cost'].sum()
        print(f"  G-series (Graphics/Inference): ${g_total:>12,.2f} ({g_total/top_account_cost*100:.1f}%)")
    if not storage_instances.empty:
        storage_total = storage_instances['cost'].sum()
        print(f"  Storage optimized (i*, d*):    ${storage_total:>12,.2f} ({storage_total/top_account_cost*100:.1f}%)")
    if not compute_instances.empty:
        compute_total = compute_instances['cost'].sum()
        print(f"  Compute optimized (c*, z*):    ${compute_total:>12,.2f} ({compute_total/top_account_cost*100:.1f}%)")
    if not general_instances.empty:
        general_total = general_instances['cost'].sum()
        print(f"  General purpose (m*, t*, r*):  ${general_total:>12,.2f} ({general_total/top_account_cost*100:.1f}%)")
else:
    print("No instance type data available.")


Cost by Instance Category:
------------------------------------------------------------
  P-series (ML Training/HPC):    $  153,300.55 (24.5%)
  G-series (Graphics/Inference): $  179,534.10 (28.7%)
  Storage optimized (i*, d*):    $  118,815.72 (19.0%)
  Compute optimized (c*, z*):    $   36,543.90 (5.9%)
  General purpose (m*, t*, r*):  $  136,374.60 (21.8%)


### 2.5.2 Monthly Trend & Fiscal Year Forecast

Cost trajectory and year-end projections for Research Platform Prod.

In [11]:
# Monthly trend for Research Platform Prod
top_monthly_df = parse_monthly_costs(top_account)

if not top_monthly_df.empty:
    fig = create_monthly_trend_chart(top_monthly_df)
    fig.update_layout(
        title=f"Monthly EC2 Cost Trend - {top_account_name}",
        height=400
    )
    fig.show()
    
    # Monthly change analysis
    if len(top_monthly_df) >= 2:
        latest = top_monthly_df.iloc[-1]['cost']
        previous = top_monthly_df.iloc[-2]['cost']
        change = ((latest - previous) / previous * 100) if previous > 0 else 0
        trend = "📈" if change > 0 else "📉"
        print(f"\nMonth-over-Month Change: {trend} {change:+.1f}%")
        
    # Show monthly breakdown
    print("\nMonthly Cost Breakdown:")
    print("-" * 50)
    for idx, row in top_monthly_df.iterrows():
        print(f"  {row['period'][:20]:20s}  ${row['cost']:>12,.2f}")
else:
    print("No monthly cost data available.")

# Fiscal Year Forecast
fy_data = top_account.get('fiscal_year_forecast', {})
if fy_data:
    print("\n" + "="*60)
    print(f"FISCAL YEAR FORECAST: {fy_data.get('fiscal_year_label', 'Unknown')}")
    print("="*60)
    print(f"\nActual Spend (YTD):      ${fy_data.get('total_actual', 0):>15,.2f}")
    print(f"Forecasted Remaining:    ${fy_data.get('total_forecast', 0):>15,.2f}")
    print(f"{'-'*50}")
    print(f"Projected FY Total:      ${fy_data.get('total_projected', 0):>15,.2f}")


Month-over-Month Change: 📉 -9.6%

Monthly Cost Breakdown:
--------------------------------------------------
  2025-09-24 to 2025-1  $   22,143.65
  2025-10-01 to 2025-1  $  229,334.32
  2025-11-01 to 2025-1  $  196,041.77
  2025-12-01 to 2025-1  $  177,152.17

FISCAL YEAR FORECAST: FY2025/2026

Actual Spend (YTD):      $   3,094,118.93
Forecasted Remaining:    $   1,064,428.97
--------------------------------------------------
Projected FY Total:      $   4,158,547.90


### 2.5.3 Optimization Opportunities

Detailed action plan for cost optimization in Research Platform Prod. This account has **$109K/month** in identified savings potential.

In [12]:
# Detailed Action Plan for Research Platform Prod
action_plan = top_account_action_plan

if action_plan:
    total_savings = action_plan.get('total_savings', 0)
    
    print("="*80)
    print(f"OPTIMIZATION ACTION PLAN - {top_account_name}")
    print("="*80)
    print(f"\nTotal Monthly Savings Potential: ${total_savings:,.2f}")
    print(f"Total Annual Savings Potential:  ${total_savings * 12:,.2f}")
    print("="*80)
    
    # Build action table
    all_actions = []
    priority_labels = {
        'immediate': ('🔴 IMMEDIATE', 1),
        'high_priority': ('🟠 HIGH', 2),
        'medium_priority': ('🟡 MEDIUM', 3)
    }
    
    for priority_key, (priority_label, sort_order) in priority_labels.items():
        actions = action_plan.get(priority_key, [])
        for action in actions:
            if isinstance(action, dict):
                all_actions.append({
                    'Priority': priority_label,
                    'Sort': sort_order,
                    'Action': action.get('title', 'N/A'),
                    'Monthly Savings': action.get('savings', 0),
                    'Annual Savings': action.get('savings', 0) * 12,
                    'Effort': action.get('effort', 'N/A'),
                    'Command': action.get('command', 'N/A')
                })
    
    if all_actions:
        actions_df = pd.DataFrame(all_actions)
        actions_df = actions_df.sort_values('Sort')
        
        # Create waterfall chart for savings
        fig = go.Figure(go.Waterfall(
            name="Savings",
            orientation="v",
            measure=["relative"] * len(all_actions) + ["total"],
            x=[a['Action'][:30] + "..." if len(a['Action']) > 30 else a['Action'] for a in all_actions] + ["Total"],
            y=[a['Monthly Savings'] for a in all_actions] + [total_savings],
            connector={"line": {"color": "rgb(63, 63, 63)"}},
            decreasing={"marker": {"color": "#2ecc71"}},
            increasing={"marker": {"color": "#2ecc71"}},
            totals={"marker": {"color": "#3498db"}}
        ))
        fig.update_layout(
            title=f"Savings Breakdown - {top_account_name}",
            showlegend=False,
            height=400,
            xaxis_tickangle=-45
        )
        fig.show()
        
        # Display action table
        print("\nDetailed Actions:")
        print("-" * 100)
        for idx, row in actions_df.iterrows():
            print(f"\n{row['Priority']}: {row['Action']}")
            print(f"   Savings: ${row['Monthly Savings']:,.2f}/month (${row['Annual Savings']:,.2f}/year)")
            print(f"   Effort: {row['Effort']}")
            print(f"   Command: {row['Command'][:80]}...")
        
        # Summary by priority
        print("\n" + "="*60)
        print("SAVINGS BY PRIORITY")
        print("="*60)
        for priority_key, (priority_label, _) in priority_labels.items():
            priority_actions = [a for a in all_actions if a['Priority'] == priority_label]
            priority_total = sum(a['Monthly Savings'] for a in priority_actions)
            if priority_total > 0:
                print(f"  {priority_label}: ${priority_total:>12,.2f}/month ({len(priority_actions)} actions)")
else:
    print("No action plan data available.")

OPTIMIZATION ACTION PLAN - AISI-Research-Platform-prod

Total Monthly Savings Potential: $106,315.43
Total Annual Savings Potential:  $1,275,785.21



Detailed Actions:
----------------------------------------------------------------------------------------------------

🔴 IMMEDIATE: Delete 34 orphaned EBS volumes
   Savings: $7,253.00/month ($87,036.00/year)
   Effort: < 1 hour
   Command: Use AWS Console or: aws ec2 delete-volume --volume-id <vol-id>...

🔴 IMMEDIATE: Stop or terminate 342 idle instances
   Savings: $17,100.00/month ($205,200.00/year)
   Effort: < 1 hour
   Command: aws ec2 stop-instances --instance-ids <instance-id>...

🟠 HIGH: Purchase recommended Savings Plans
   Savings: $21,862.43/month ($262,349.21/year)
   Effort: < 1 day
   Command: Review recommendations in AWS Cost Explorer → Savings Plans...

🟡 MEDIUM: Migrate gp2 volumes to gp3
   Savings: $80.00/month ($960.00/year)
   Effort: < 1 week
   Command: aws ec2 modify-volume --volume-id <vol-id> --volume-type gp3...

🟡 MEDIUM: Migrate 584 instances to Spot
   Savings: $58,400.00/month ($700,800.00/year)
   Effort: < 1 month
   Command: Use EC2 Spot Instances 

### 2.5.4 Key Insights & Recommendations

**Top Spending Drivers:**
- **P-series instances** (p5e, p4d, p5) - ML training and HPC workloads
- **G-series instances** (g6, g5) - Graphics rendering and ML inference
- **Storage-optimized** (i4i.metal) - Heavy data processing requirements

**Quick Wins (Implementation: < 1 day):**
1. **Delete orphaned EBS volumes** - 34 unattached volumes costing ~$7.3K/month
2. **Address idle instances** - 352 instances with <5% CPU utilization (~$17.6K/month)

**Strategic Optimizations:**
1. **Savings Plans** - $21.9K/month potential with 1-year commitments on consistent workloads

**Note on Spot Instances:** The analyzer identified 604 instances as spot-eligible (~$60.4K/month potential). However, **spot migration is NOT recommended for production workloads** due to interruption risk. Consider spot instances only for:
- Batch processing jobs
- Development/test environments
- Fault-tolerant workloads with checkpointing

**Additional Analysis:** For tag breakdowns (Environment, Project, Owner) and CloudWatch utilization metrics, re-run:
```bash
uv run python multi_account_analyzer.py
```

---
# Section 3: Optimization Potential by Account

Savings opportunities identified across all accounts.

In [13]:
# Multi-account savings waterfall
total_optimization = aggregated.get('total_optimization_potential', 0)

print("\n" + "="*80)
print("OPTIMIZATION POTENTIAL BY ACCOUNT")
print("="*80)
print(f"\nTotal Monthly Optimization Potential: ${total_optimization:,.2f}")
print(f"Total Annual Optimization Potential: ${total_optimization * 12:,.2f}")
print("="*80)

fig = create_multi_account_savings_waterfall(data, "Monthly Optimization Potential by Account")
fig.show()


OPTIMIZATION POTENTIAL BY ACCOUNT

Total Monthly Optimization Potential: $115,209.48
Total Annual Optimization Potential: $1,382,513.78


In [14]:
# Summary table of savings by account
account_reports = data.get('account_reports', {})
savings_summary = []

for account_id, account_data in account_reports.items():
    account_name = account_data.get('account_name', account_id)
    total_cost = account_data.get('total_cost', 0)
    action_plan = account_data.get('action_plan', {})
    total_savings = action_plan.get('total_savings', 0)
    
    savings_summary.append({
        'Account': account_name,
        'Total Cost (3-mo)': total_cost,
        'Monthly Savings': total_savings,
        'Annual Savings': total_savings * 12,
        'Savings %': (total_savings / (total_cost / 3) * 100) if total_cost > 0 else 0
    })

savings_df = pd.DataFrame(savings_summary)
savings_df = savings_df.sort_values('Monthly Savings', ascending=False)

print("\nSavings Potential by Account:\n")
display_df = savings_df.copy()
display_df['Total Cost (3-mo)'] = display_df['Total Cost (3-mo)'].apply(lambda x: f"${x:,.2f}")
display_df['Monthly Savings'] = display_df['Monthly Savings'].apply(lambda x: f"${x:,.2f}")
display_df['Annual Savings'] = display_df['Annual Savings'].apply(lambda x: f"${x:,.2f}")
display_df['Savings %'] = display_df['Savings %'].apply(lambda x: f"{x:.1f}%")
display(display_df)


Savings Potential by Account:



,Account,Total Cost (3-mo),Monthly Savings,Annual Savings,Savings %
1,AISI-Research-Platform-prod,"$624,671.91","$106,315.43","$1,275,785.21",51.1%
0,AISI-Research-Platform-preprod,"$13,930.89","$4,515.14","$54,181.68",97.2%
2,AISI-Research-Platform-dev,"$5,973.28","$2,958.68","$35,504.11",148.6%
6,AISIWADevelopment,$25.48,$487.89,"$5,854.73",5744.5%
4,AISIWAProduction,$231.46,$473.14,"$5,677.66",613.2%
5,AISI-Self-Hosting-prod,"$56,652.02",$292.01,"$3,504.12",1.5%
3,AISIWAStaging,$0.01,$167.19,"$2,006.26",8327853.4%


---
# Section 4: Per-Account Drill-Down

Select an account to view detailed analysis including resource utilization, storage optimization, and specific recommendations.

In [15]:
# Account selector dropdown
account_list = get_account_list(data)
account_options = [(f"{acc['account_name']} (${acc['total_cost']:,.0f})", acc['account_id']) for acc in account_list]

account_dropdown = widgets.Dropdown(
    options=account_options,
    description='Select Account:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

output = widgets.Output()

def display_account_details(account_id):
    """Display detailed analysis for selected account."""
    account_data = get_account_data(data, account_id)
    account_name = account_data.get('account_name', account_id)
    
    print("\n" + "="*80)
    print(f"DETAILED ANALYSIS: {account_name}")
    print(f"Account ID: {account_id}")
    print("="*80)
    
    # Summary
    total_cost = account_data.get('total_cost', 0)
    action_plan = account_data.get('action_plan', {})
    total_savings = action_plan.get('total_savings', 0)
    
    print(f"\nTotal EC2 Cost (3-month): ${total_cost:,.2f}")
    print(f"Optimization Potential: ${total_savings:,.2f}/month")
    
    # Monthly trend
    print("\n--- Monthly Cost Trend ---")
    monthly_df = parse_monthly_costs(account_data)
    if not monthly_df.empty:
        fig = create_monthly_trend_chart(monthly_df)
        fig.update_layout(title=f"Monthly Cost Trend - {account_name}")
        fig.show()
    
    # Instance types
    print("\n--- Top Instance Types ---")
    instance_df = parse_instance_type_costs(account_data, top_n=10)
    if not instance_df.empty:
        fig = create_horizontal_bar_chart(instance_df, 'cost', 'instance_type',
                                           f'Top 10 Instance Types - {account_name}', 'Cost ($)')
        fig.show()
    
    # Cloudwatch analysis (idle/underutilized)
    cw_analysis = account_data.get('cloudwatch_analysis', {})
    idle_instances = cw_analysis.get('idle', [])
    underutil_instances = cw_analysis.get('underutilized', [])
    
    if idle_instances:
        print(f"\n--- Idle Instances ({len(idle_instances)} found) ---")
        idle_df = pd.DataFrame(idle_instances)
        if 'InstanceId' in idle_df.columns:
            display(idle_df[['InstanceId', 'InstanceType', 'AvgCPU']].head(10))
    
    if underutil_instances:
        print(f"\n--- Underutilized Instances ({len(underutil_instances)} found) ---")
        underutil_df = pd.DataFrame(underutil_instances)
        if 'InstanceId' in underutil_df.columns:
            display(underutil_df[['InstanceId', 'InstanceType', 'AvgCPU']].head(10))
    
    # EBS Analysis
    ebs_data = account_data.get('ebs_analysis', {})
    if ebs_data:
        print("\n--- Storage Optimization ---")
        print(f"  Unattached volumes: {ebs_data.get('unattached_count', 0)}")
        print(f"  Unattached savings: ${ebs_data.get('unattached_savings', 0):,.2f}/month")
        print(f"  GP2->GP3 migration savings: ${ebs_data.get('gp2_migration_savings', 0):,.2f}/month")
    
    # RI/SP Analysis
    ri_sp = account_data.get('ri_sp_analysis', {})
    if ri_sp:
        print("\n--- Savings Plans / Reserved Instances ---")
        print(f"  Current coverage: {ri_sp.get('coverage', 0):.1f}%")
        print(f"  Potential savings: ${ri_sp.get('potential_savings', 0):,.2f}/month")
    
    # Action Plan
    if action_plan:
        print("\n--- Action Plan Summary ---")
        print(f"  Total monthly savings: ${total_savings:,.2f}")
        
        for priority in ['immediate', 'high_priority', 'medium_priority']:
            actions = action_plan.get(priority, [])
            if actions:
                print(f"\n  {priority.replace('_', ' ').title()}:")
                for action in actions[:3]:  # Show top 3 per priority
                    if isinstance(action, dict):
                        title = action.get('title', action.get('action', 'N/A'))
                        print(f"    - {title}: ${action.get('savings', 0):,.2f}/mo")

def on_account_change(change):
    with output:
        clear_output(wait=True)
        display_account_details(change['new'])

account_dropdown.observe(on_account_change, names='value')

print("Select an account to view detailed analysis:")
display(account_dropdown)
display(output)

# Trigger initial display
if account_options:
    with output:
        display_account_details(account_options[0][1])

Select an account to view detailed analysis:


Dropdown(description='Select Account:', layout=Layout(width='400px'), options=(('AISI-Research-Platform-prod (…

Output()

---
# Section 5: Key Recommendations Summary

Top actionable items across all accounts.

In [16]:
# Aggregate recommendations from all accounts
all_recommendations = []

for account_id, account_data in account_reports.items():
    account_name = account_data.get('account_name', account_id)
    action_plan = account_data.get('action_plan', {})
    
    for priority in ['immediate', 'high_priority', 'medium_priority']:
        actions = action_plan.get(priority, [])
        for action in actions:
            if isinstance(action, dict):
                title = action.get('title', action.get('action', 'N/A'))
                all_recommendations.append({
                    'Account': account_name,
                    'Priority': priority.replace('_', ' ').title(),
                    'Action': title,
                    'Monthly Savings': action.get('savings', 0),
                    'Effort': action.get('effort', 'N/A')
                })

if all_recommendations:
    rec_df = pd.DataFrame(all_recommendations)
    rec_df = rec_df.sort_values('Monthly Savings', ascending=False)
    
    print("\n" + "="*80)
    print("TOP 15 RECOMMENDATIONS ACROSS ALL ACCOUNTS (Ranked by Savings)")
    print("="*80 + "\n")
    
    for i, (idx, row) in enumerate(rec_df.head(15).iterrows(), 1):
        print(f"{i}. [{row['Priority']}] {row['Action']}")
        print(f"   Account: {row['Account']}")
        print(f"   Savings: ${row['Monthly Savings']:,.2f}/month | Effort: {row['Effort']}")
        print()
else:
    print("No recommendations available.")


TOP 15 RECOMMENDATIONS ACROSS ALL ACCOUNTS (Ranked by Savings)

1. [Medium Priority] Migrate 584 instances to Spot
   Account: AISI-Research-Platform-prod
   Savings: $58,400.00/month | Effort: < 1 month

2. [High Priority] Purchase recommended Savings Plans
   Account: AISI-Research-Platform-prod
   Savings: $21,862.43/month | Effort: < 1 day

3. [Immediate] Stop or terminate 342 idle instances
   Account: AISI-Research-Platform-prod
   Savings: $17,100.00/month | Effort: < 1 hour

4. [Immediate] Delete 34 orphaned EBS volumes
   Account: AISI-Research-Platform-prod
   Savings: $7,253.00/month | Effort: < 1 hour

5. [Medium Priority] Migrate 26 instances to Spot
   Account: AISI-Research-Platform-preprod
   Savings: $2,600.00/month | Effort: < 1 month

6. [Immediate] Delete 2770 orphaned EBS volumes
   Account: AISI-Research-Platform-dev
   Savings: $1,832.90/month | Effort: < 1 hour

7. [Immediate] Stop or terminate 21 idle instances
   Account: AISI-Research-Platform-preprod
   Sav

---
# Export Options

Export data for implementation and tracking.

In [17]:
# CSV Export Functions (run manually when needed)

def export_account_summary():
    """Export account cost summary to CSV."""
    account_summary = []
    for acc in get_account_list(data):
        account_data = get_account_data(data, acc['account_id'])
        action_plan = account_data.get('action_plan', {})
        account_summary.append({
            'Account ID': acc['account_id'],
            'Account Name': acc['account_name'],
            'Total Cost (3-mo)': acc['total_cost'],
            'Monthly Savings Potential': action_plan.get('total_savings', 0)
        })
    
    summary_df = pd.DataFrame(account_summary)
    csv_filename = f"multi_account_summary_{datetime.now().strftime('%Y%m%d')}.csv"
    summary_df.to_csv(csv_filename, index=False)
    print(f"Account summary exported to: {csv_filename}")
    return csv_filename

def export_recommendations():
    """Export all recommendations to CSV."""
    all_recs = []
    account_reports = data.get('account_reports', {})
    
    for account_id, account_data in account_reports.items():
        account_name = account_data.get('account_name', account_id)
        action_plan = account_data.get('action_plan', {})
        
        for priority in ['immediate', 'high_priority', 'medium_priority']:
            actions = action_plan.get(priority, [])
            for action in actions:
                if isinstance(action, dict):
                    title = action.get('title', action.get('action', 'N/A'))
                    all_recs.append({
                        'Account': account_name,
                        'Priority': priority.replace('_', ' ').title(),
                        'Action': title,
                        'Monthly Savings': action.get('savings', 0),
                        'Effort': action.get('effort', 'N/A')
                    })
    
    if all_recs:
        rec_df = pd.DataFrame(all_recs)
        rec_filename = f"multi_account_recommendations_{datetime.now().strftime('%Y%m%d')}.csv"
        rec_df.to_csv(rec_filename, index=False)
        print(f"Recommendations exported to: {rec_filename}")
        return rec_filename
    else:
        print("No recommendations to export.")
        return None

print("Export functions available:")
print("  - export_account_summary()  : Export account costs to CSV")
print("  - export_recommendations()  : Export all recommendations to CSV")
print("\nTo export notebook as HTML:")
print("  jupyter nbconvert --to html ec2_cost_optimization_report.ipynb")

Export functions available:
  - export_account_summary()  : Export account costs to CSV
  - export_recommendations()  : Export all recommendations to CSV

To export notebook as HTML:
  jupyter nbconvert --to html ec2_cost_optimization_report.ipynb


---
# Report Complete

**Next Steps:**
1. Review the optimization potential by account
2. Use the drill-down section to analyze specific accounts
3. Prioritize actions by savings potential
4. Execute immediate actions first (orphaned volumes, idle instances)
5. Re-run this report weekly/monthly to track progress

**To re-run with fresh data:**
```bash
# Step 1: Ensure SSO session is active
aws sso login --sso-session cost-analyzer-setup

# Step 2: Generate new multi-account cost report
uv run python multi_account_analyzer.py

# Step 3: Run all cells (Kernel -> Restart & Run All)
```